## 使用pymysql与数据库交互
1. 建立连接
2. 创建游标
3. 执行操作
4. 关闭连接

In [1]:
import pymysql
import time 
import json
import traceback #追踪异常 

In [2]:
def get_tencent_data():
    '''
    return：返回历史数据和当日详情数据
    '''
    url = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
    headers = {
        "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
    }
    r = requests.get(url,headers)
    res = json.loads(r.text)#json字符串转字典
    data_all = jason.loads(res["data"])
    
    history={}#历史数据
    for i in data_all["chinaDayList"]:
        ds = "2020." + i["date"]
        tup = time.strptime(ds,"%Y.%m.%d")
        ds = time.strftime("%Y-%m-%d",tup)#改变时间格式否则插入数据库报错
        confirm = i["confirm"]
        suspect = i["suspect"]
        heal = i["heal"]
        dead = i["dead"]
        history[ds] = {"confirm":confirm,"suspect":suspect,"heal":heal,"dead":dead}
    for i in data_all["chinaAddList"]:
        ds = "2020." + i["date"]
        tup = time.strptime(ds,"%Y.%m.%d")
        ds = time.strftime("%Y-%m-%d",tup)#改变时间格式否则插入数据库报错
        confirm = i["confirm"]
        suspect = i["suspect"]
        heal = i["heal"]
        dead = i["dead"]
        history[ds].updata({"confirm_add":confirm,"suspect_add":suspect,"heal_add":heal,"dead_add":dead})
        
    details=[]#当日详情数据
    update_time = data_all["lastUpdateTime"]
    data_country = data_all["areaTree"]#list25个国家
    data_province = data_country[0]["children"]#中国各省
    for pro_infos in data_province:
        province = pro_infos["name"]#省名
        for city_infos in pro_infos["children"]:
            city = city_infos["name"]
            confirm = city_infos["total"]["confirm"]
            confirm_add = city_infos["today"]["confirm"]
            heal = city_infos["total"]["heal"]
            head = city_infos["total"]["head"]
            details.append([update_time,province,city,confirm,confirm_add,heal,head])
    return history,details

### 存储腾讯数据

### pymysql的简单使用

In [3]:

#建立连接
conn = pymysql.connect(host="122.51.251.91",user="user1",password="123456",db="cov")
#创建游标，默认为元组类型
cursor = conn.cursor()

sql="select * from history"#查询history的所有数据
cursor.execute(sql)

#conn.commit() #提交事务
res = cursor.fetchall()#fetchall方法获得查询结果
print(res)

cursor.close()
conn.close()


OperationalError: (1045, "Access denied for user 'user1'@'183.202.37.221' (using password: YES)")